In [1]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.metrics import f1_score, make_scorer, precision_score, recall_score, accuracy_score, confusion_matrix
from scipy.sparse import hstack, csr_matrix
import re
from collections import defaultdict, Counter
from nltk import TweetTokenizer, pos_tag
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
from IPython.display import display
import os
import numpy as np

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, load_model
from keras.layers import Dense, Conv1D, MaxPooling1D, Embedding, Flatten, Dropout
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from keras import optimizers

from IPython.core.debugger import set_trace

/usr0/home/mamille2/anaconda3/lib/python3.6/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "
Using TensorFlow backend.


In [4]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
# Load data
data = {}
# data['train'] = pd.read_csv('../../hw3/train_mod.csv')
data['train'] = pd.read_csv('/usr0/home/mamille2/11-830_data/hw3/train_mod.csv')
data['train']

,Unnamed: 0,tweet,hate,offensive,neither,tweet_processed
0,0,"@HarveyLevinTMZ @sly309 they want what I got, ...",1,0,0,"<mention> <mention> they want what i got , i w..."
1,1,@HeauxmerSimpson I'm jus tryna vaca away from ...,1,0,0,<mention> i'm jus tryna vaca away from the nig...
2,2,@HeilSidious Sion is a faggot. Very meh ganks ...,1,0,0,<mention> sion is a faggot . very meh ganks pr...
3,3,@Herfarm @SteveWorks4You @TheDemocrats your lo...,1,0,0,<mention> <mention> <mention> your low info re...
4,4,@Herfarm Move out of our country teabaggers!,1,0,0,<mention> move out of our country teabaggers !
5,5,@HollowDaDonLOM those god damn chinks. http://...,1,0,0,<mention> those god damn chinks . http://t.co/...
6,6,@Hovaa_ #niggerfood #niggerperson #you #are #a...,1,0,0,<mention> #niggerfood #niggerperson #you #are ...
7,7,@Hovaa_ ok wat ever u say whitey. u prolly nev...,1,0,0,<mention> ok wat ever u say whitey . u prolly ...
8,8,@Hovaa_ shut up lizard faggot nigger cunt,1,0,0,<mention> shut up lizard faggot nigger cunt
9,9,@Hovaa_ ya I know all the slang I'm racist I h...,1,0,0,<mention> ya i know all the slang i'm racist i...


In [3]:
# Load test data
# data['train'] = pd.read_csv('../../hw3/train_mod.csv')
data['test'] = pd.read_csv('/usr0/home/mamille2/11-830_data/hw3/test_comments.csv')
data['test']

,ID,tweet,tweet_processed
0,0,!!! RT @mayasolovely: As a woman you shouldn't...,! ! ! rt <mention> : as a woman you shouldn't ...
1,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,! ! ! rt <mention> : boy dats cold ... tyga dw...
2,2,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,! ! ! rt <mention> dawg ! ! ! rt <mention> : y...
3,3,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,! ! ! rt <mention> : <mention> she look like a...
4,4,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,! ! ! rt <mention> : the shit you hear about m...
5,5,"!!!!!!!!!!!!!!!!!!""@T_Madison_x: The shit just...","! ! ! "" <mention> : the shit just blows me .. ..."
6,6,"!!!!!!""@__BrighterDays: I can not just sit up ...","! ! ! "" <mention> : i can not just sit up and ..."
7,7,!!!!&#8220;@selfiequeenbri: cause I'm tired of...,! ! ! “ <mention> : cause i'm tired of you big...
8,8,""" &amp; you might not get ya bitch back &amp; ...",""" & you might not get ya bitch back & thats th..."
9,9,""" @rhythmixx_ :hobbies include: fighting Maria...",""" <mention> : hobbies include : fighting maria..."


## Preprocess

In [6]:
def preprocess_tweet(text):
    
    # Remove @mentions
    p = re.sub(r'@\w+', '<mention>', text)
    
    # tokenize
    tokenizer = TweetTokenizer()
    p = ' '.join(tokenizer.tokenize(p.lower()))
    
    return p

In [4]:
data['train']['tweet_processed'] = data['train']['tweet'].map(lambda x: preprocess_tweet(x))

In [8]:
data['test']['tweet_processed'] = data['test']['tweet'].map(lambda x: preprocess_tweet(x))

In [41]:
data['train']['tweet_processed'].head()

0    <mention> <mention> they want what i got , i want what they got , america.they got the word nigger , i want it to                           
1    <mention> i'm jus tryna vaca away from the niggers bro .                                                                                    
2    <mention> sion is a faggot . very meh ganks pre - 6 and post - 6 his ganks are still pretty meh he's just a tank , nothing special imo mao +
3    <mention> <mention> <mention> your low info redneck dumbasses vote for these corporate shills that idiots like you believe #dummies         
4    <mention> move out of our country teabaggers !                                                                                              
Name: tweet_processed, dtype: object

In [42]:
data['train'].to_csv('../../hw3/train_mod.csv')

In [10]:
# data['test'].to_csv('../../hw3/test_comments.csv')
data['test'].to_csv('/usr0/home/mamille2/11-830_data/hw3/test_comments.csv', index=False)

# Logistic regression baseline

In [4]:
vec = CountVectorizer(min_df=2)
vec.fit(data['train'].tweet_processed.tolist())

bow = {}
bow['train'] = vec.transform(data['train'].tweet_processed.tolist())
bow['train'].shape

(21083, 8534)

In [6]:
def f1(prec, rec):
    return 2*prec*rec / (prec + rec)

In [7]:
# Hate speech classification
x = {}
y = {}
x['train'] = bow['train']
y['train'] = data['train'].hate

clf = LogisticRegression()

scoring = {'accuracy': make_scorer(accuracy_score), 
           'precision': make_scorer(precision_score),
          'recall': make_scorer(recall_score),
          'f1': make_scorer(f1_score)}

scores = cross_validate(clf, x['train'], y['train'], cv=5, scoring=scoring)
avg_prec = scores["test_precision"].mean()
avg_rec = scores["test_recall"].mean()

print(f'Accuracy: {scores["test_accuracy"].mean(): .1%}')
print(f'Precision: {avg_prec: .1%}')
print(f'Recall: {avg_rec: .1%}')
# print(f'F1: {scores["test_f1"].mean(): .1%}')
print(f'F1: {f1(avg_prec, avg_rec): .1%}')

Accuracy:  93.8%
Precision:  48.5%
Recall:  21.3%
F1:  29.6%


In [8]:
# Fit classifier to all training data
clf.fit(x['train'], y['train'])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [9]:
# Get confusion matrix
predicted = clf.predict(x['train'])
confusion_matrix(y['train'], predicted)

array([[19804,    63],
       [  695,   521]])

In [14]:
# Make predictions on test set

bow['test'] = vec.transform(data['test'].tweet_processed.tolist())
print(bow['test'].shape)

data['test']['hate'] = clf.predict(bow['test'])
data['test']

(3700, 8534)


,ID,tweet,tweet_processed,hate
0,0,!!! RT @mayasolovely: As a woman you shouldn't...,! ! ! rt <mention> : as a woman you shouldn't ...,0
1,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,! ! ! rt <mention> : boy dats cold ... tyga dw...,0
2,2,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,! ! ! rt <mention> dawg ! ! ! rt <mention> : y...,0
3,3,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,! ! ! rt <mention> : <mention> she look like a...,0
4,4,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,! ! ! rt <mention> : the shit you hear about m...,0
5,5,"!!!!!!!!!!!!!!!!!!""@T_Madison_x: The shit just...","! ! ! "" <mention> : the shit just blows me .. ...",0
6,6,"!!!!!!""@__BrighterDays: I can not just sit up ...","! ! ! "" <mention> : i can not just sit up and ...",0
7,7,!!!!&#8220;@selfiequeenbri: cause I'm tired of...,! ! ! “ <mention> : cause i'm tired of you big...,0
8,8,""" &amp; you might not get ya bitch back &amp; ...",""" & you might not get ya bitch back & thats th...",0
9,9,""" @rhythmixx_ :hobbies include: fighting Maria...",""" <mention> : hobbies include : fighting maria...",0


In [11]:
# Offensive language classification
x = {}
y = {}
x['train'] = bow['train']
y['train'] = data['train'].offensive

clf = LogisticRegression()

scoring = {'accuracy': make_scorer(accuracy_score), 
           'precision': make_scorer(precision_score),
          'recall': make_scorer(recall_score),
          'f1': make_scorer(f1_score)}

scores = cross_validate(clf, x['train'], y['train'], cv=5, scoring=scoring)
avg_prec = scores["test_precision"].mean()
avg_rec = scores["test_recall"].mean()

print(f'Accuracy: {scores["test_accuracy"].mean(): .1%}')
print(f'Precision: {avg_prec: .1%}')
print(f'Recall: {avg_rec: .1%}')
# print(f'F1: {scores["test_f1"].mean(): .1%}')
print(f'F1: {f1(avg_prec, avg_rec): .1%}')

Accuracy:  89.2%
Precision:  92.8%
Recall:  93.3%
F1:  93.0%


In [12]:
# Fit classifier to all training data
clf = LogisticRegression()
clf.fit(x['train'], y['train'])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [13]:
# Get confusion matrix
predicted = clf.predict(x['train'])
confusion_matrix(y['train'], predicted)

array([[ 4125,   633],
       [  477, 15848]])

In [20]:
# Make predictions on test set

bow['test'] = vec.transform(data['test'].tweet_processed.tolist())
print(bow['test'].shape)

data['test']['offensive'] = clf.predict(bow['test'])
data['test']

(3700, 8534)


,ID,tweet,tweet_processed,hate,offensive
0,0,!!! RT @mayasolovely: As a woman you shouldn't...,! ! ! rt <mention> : as a woman you shouldn't ...,0,1
1,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,! ! ! rt <mention> : boy dats cold ... tyga dw...,0,1
2,2,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,! ! ! rt <mention> dawg ! ! ! rt <mention> : y...,0,1
3,3,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,! ! ! rt <mention> : <mention> she look like a...,0,0
4,4,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,! ! ! rt <mention> : the shit you hear about m...,0,1
5,5,"!!!!!!!!!!!!!!!!!!""@T_Madison_x: The shit just...","! ! ! "" <mention> : the shit just blows me .. ...",0,1
6,6,"!!!!!!""@__BrighterDays: I can not just sit up ...","! ! ! "" <mention> : i can not just sit up and ...",0,1
7,7,!!!!&#8220;@selfiequeenbri: cause I'm tired of...,! ! ! “ <mention> : cause i'm tired of you big...,0,1
8,8,""" &amp; you might not get ya bitch back &amp; ...",""" & you might not get ya bitch back & thats th...",0,1
9,9,""" @rhythmixx_ :hobbies include: fighting Maria...",""" <mention> : hobbies include : fighting maria...",0,1


In [22]:
data['test']['neither'] = [1 if not a and not b else 0 for a,b in zip(data['test']['hate'], data['test']['offensive'])]
data['test']

,ID,tweet,tweet_processed,hate,offensive,neither
0,0,!!! RT @mayasolovely: As a woman you shouldn't...,! ! ! rt <mention> : as a woman you shouldn't ...,0,1,0
1,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,! ! ! rt <mention> : boy dats cold ... tyga dw...,0,1,0
2,2,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,! ! ! rt <mention> dawg ! ! ! rt <mention> : y...,0,1,0
3,3,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,! ! ! rt <mention> : <mention> she look like a...,0,0,1
4,4,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,! ! ! rt <mention> : the shit you hear about m...,0,1,0
5,5,"!!!!!!!!!!!!!!!!!!""@T_Madison_x: The shit just...","! ! ! "" <mention> : the shit just blows me .. ...",0,1,0
6,6,"!!!!!!""@__BrighterDays: I can not just sit up ...","! ! ! "" <mention> : i can not just sit up and ...",0,1,0
7,7,!!!!&#8220;@selfiequeenbri: cause I'm tired of...,! ! ! “ <mention> : cause i'm tired of you big...,0,1,0
8,8,""" &amp; you might not get ya bitch back &amp; ...",""" & you might not get ya bitch back & thats th...",0,1,0
9,9,""" @rhythmixx_ :hobbies include: fighting Maria...",""" <mention> : hobbies include : fighting maria...",0,1,0


In [25]:
# Select hate speech if both hate speech and offensive lang predicted
both_idx = data['test'][(data['test']['hate']==1) & (data['test']['offensive'])].index
len(both_idx)

0

In [24]:
for i in both_idx:
    data['test'].loc[i, 'offensive'] = 0

In [26]:
data['test']

,ID,tweet,tweet_processed,hate,offensive,neither
0,0,!!! RT @mayasolovely: As a woman you shouldn't...,! ! ! rt <mention> : as a woman you shouldn't ...,0,1,0
1,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,! ! ! rt <mention> : boy dats cold ... tyga dw...,0,1,0
2,2,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,! ! ! rt <mention> dawg ! ! ! rt <mention> : y...,0,1,0
3,3,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,! ! ! rt <mention> : <mention> she look like a...,0,0,1
4,4,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,! ! ! rt <mention> : the shit you hear about m...,0,1,0
5,5,"!!!!!!!!!!!!!!!!!!""@T_Madison_x: The shit just...","! ! ! "" <mention> : the shit just blows me .. ...",0,1,0
6,6,"!!!!!!""@__BrighterDays: I can not just sit up ...","! ! ! "" <mention> : i can not just sit up and ...",0,1,0
7,7,!!!!&#8220;@selfiequeenbri: cause I'm tired of...,! ! ! “ <mention> : cause i'm tired of you big...,0,1,0
8,8,""" &amp; you might not get ya bitch back &amp; ...",""" & you might not get ya bitch back & thats th...",0,1,0
9,9,""" @rhythmixx_ :hobbies include: fighting Maria...",""" <mention> : hobbies include : fighting maria...",0,1,0


In [28]:
baseline_predictions = data['test'].drop('tweet_processed', axis=1, inplace=False)
baseline_predictions

,ID,tweet,hate,offensive,neither
0,0,!!! RT @mayasolovely: As a woman you shouldn't...,0,1,0
1,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,0,1,0
2,2,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,0,1,0
3,3,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,0,0,1
4,4,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,0,1,0
5,5,"!!!!!!!!!!!!!!!!!!""@T_Madison_x: The shit just...",0,1,0
6,6,"!!!!!!""@__BrighterDays: I can not just sit up ...",0,1,0
7,7,!!!!&#8220;@selfiequeenbri: cause I'm tired of...,0,1,0
8,8,""" &amp; you might not get ya bitch back &amp; ...",0,1,0
9,9,""" @rhythmixx_ :hobbies include: fighting Maria...",0,1,0


In [29]:
# Save predictions
baseline_predictions.to_csv('/usr0/home/mamille2/11-830_data/hw3/Michael_Yoder.baseline', index=False)

## Error analysis (hate speech)

In [15]:
# Train on full dataset (hate speech) and see errors
clf = LogisticRegression()
y['train'] = data['train'].hate
clf.fit(x['train'], y['train'])
preds = clf.predict(x['train'])
preds.shape

(21083,)

In [16]:
false_pos = []
false_neg = []

for i, (gold, pred) in enumerate(zip(data['train'].hate.tolist(), preds)):
    if pred != gold:
        if gold == 0:
            false_pos.append(i)
        elif gold == 1:
            false_neg.append(i)
        else:
            "What???"
            
print(len(false_pos))
print(len(false_neg))

63
695


In [26]:
pd.set_option('display.max_colwidth', -1)

In [25]:
print("False negatives:")
display(data['train'].loc[false_neg, ['tweet_processed', 'hate', 'offensive', 'neither']].sample(30))

False negatives:


,tweet_processed,hate,offensive,neither
19933,"bitch kill yoself , go on to the bathroom and eat the pills bitch , all of em",1,0,0
10717,rt <mention> : <mention> <mention> <mention> not about to shoot this pussy ! ima beat his face into the asphalt t ...,1,0,0
12006,rt <mention> : the fox says you're a cunt .,1,0,0
738,<mention> fucking coon,1,0,0
3340,<mention> <mention> <mention> <mention> yeah ok nigger .,1,0,0
13655,rt <mention> : <mention> happy birthday my little cotton picker,1,0,0
14387,rt <mention> : i saw a great chimpout 2 summers ago . a middle aged salvadoran and his 15 yr old son got wasted by 6 beaners at the metlife …,1,0,0
2811,<mention> faggot,1,0,0
3255,<mention> i'm a faggot :/ http://t.co/gr627ldfaa,1,0,0
4865,curse avi for coke because he's a fag and hasn't put it on yet http://t.co/jhqpwop4wk,1,0,0


## Error analysis (offensive language)

In [28]:
# Train on full dataset and see errors
clf = LogisticRegression()
y['train'] = data['train'].offensive
clf.fit(x['train'], y['train'])
preds = clf.predict(x['train'])
preds.shape

(21083,)

In [30]:
false_pos = []
false_neg = []

for i, (gold, pred) in enumerate(zip(data['train'].offensive.tolist(), preds)):
    if pred != gold:
        if gold == 0:
            false_pos.append(i)
        elif gold == 1:
            false_neg.append(i)
        else:
            "What???"
            
print(len(false_pos))
print(len(false_neg))

633
477


In [31]:
print("False negatives:")
display(data['train'].loc[false_neg, ['tweet_processed', 'hate', 'offensive', 'neither']].sample(30))

False negatives:


,tweet_processed,hate,offensive,neither
16690,rt <mention> : <mention> blaxican,0,1,0
15079,rt <mention> : 😂 😂 rt <mention> : lol rt <mention> : 😂 😭 rt <mention> : just so u faggots know ! http://t.co/bghtsjipne,0,1,0
15728,rt <mention> : bruh you trash take yo ass back to college <mention>,0,1,0
6141,"hmm .. ass all chunky , brain is insanity .",0,1,0
12632,rt <mention> : i don't understand how so many americans can hate on obama for saluting with a latte . nobody tells these swagfags anything abo …,0,1,0
12728,rt <mention> : one man's trash is another man's girlfriend .,0,1,0
6373,"i been way to consistent on niggas , i'm have to drop some trash to get appreciated",0,1,0
15955,rt <mention> : i believe the time for uncouth / uncivilised people to be sentenced to death for racial slurs / attacks against ne citizens ...,0,1,0
11039,rt <mention> : if you don't make her go to sleep after y'all fucked then your pipe game trash,0,1,0
17480,stop spoiling u fags,0,1,0


# CNN

In [11]:
MAX_VOCAB_SIZE = 30000
tokenizer = Tokenizer(num_words=MAX_VOCAB_SIZE,
                     filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n“”')
texts = data['train']['tweet_processed'].tolist()
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
print(f'Found {len(word_index)} unique words')

Found 21870 unique words


In [54]:
# labels = data['train']['hate'].data
labels = to_categorical(data['train']['hate'].tolist())
labels.shape

(21083, 2)

In [26]:
MAX_SEQUENCE_LENGTH = 100
seqs = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
seqs.shape

(21083, 100)

In [35]:
vocab = list(word_index.keys())[:MAX_VOCAB_SIZE] # lower indices are words kept
len(vocab)

21870

In [55]:
# Shuffle, split into train/dev
test_size = int(0.1 * len(seqs))
x_train, x_dev, y_train, y_dev = train_test_split(seqs, np.asarray(labels), test_size=test_size)
print(x_train.shape)
print(y_train.shape)
print(x_dev.shape)
print(y_dev.shape)

(18975, 100)
(18975, 2)
(2108, 100)
(2108, 2)


In [61]:
p = np.array([[0.2, 0.3], [0.7, 0.3]])
np.argmax(p, axis=1)

array([1, 0])

In [71]:
metrics(p, np.array([1,0]))

Precision: 1.0
Recall: 1.0
F1: 1.0
Accuracy: 1.0


In [79]:
def metrics(preds, actual):
    tp = 0
    fp = 0
    fn = 0
    matches = 0
    
    binary_preds = np.argmax(preds, axis=1)
    binary_actual = np.argmax(actual, axis=1)
    
    for pred, act in zip(binary_preds, binary_actual):
        
        if pred == act == 1:
            tp += 1
            matches += 1
        elif pred == 1 and act == 0:
            fp += 1
        elif pred == 0 and act == 1:
            fn += 1
        elif pred == act == 0:
            matches += 1
        
        
    if tp == fp == 0:
        prec = 0
    else:
        prec = tp/(tp+fp)
        
    rec = tp/(tp+fn)
    
    if prec == rec == 0.0:
        f1 = 0.0
        
    else:
        f1 = 2 * prec * rec / (prec + rec)
        
    acc = matches/len(preds)
    
    print(f"Precision: {prec}")
    print(f"Recall: {rec}")
    print(f"F1: {f1}")
    print(f"Accuracy: {acc}")

In [86]:
# Prepare model

EMBEDDING_DIM = 50
embedding_layer = Embedding(len(vocab),
                            EMBEDDING_DIM,
                            input_length = MAX_SEQUENCE_LENGTH,
                            trainable=True
                           )

model = Sequential()

model.add(embedding_layer)
model.add(Conv1D(32, kernel_size=5, activation='relu'))
model.add(MaxPooling1D(pool_size=5))
model.add(Dropout(0.1))
model.add(Conv1D(16, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=3))
model.add(Dropout(0.1))
model.add(Flatten())
model.add(Dense(4, activation='relu')) # final classification layer
model.add(Dense(2, activation='softmax')) # final classification layer

# adam = optimizers.adam(lr=0.1)
sgd = optimizers.SGD(lr=0.05)
model.compile(loss='binary_crossentropy', optimizer=sgd)

callbacks = [
    EarlyStopping(monitor='val_loss', patience=2, verbose=0),
]
model.fit(x_train, y_train,
#          batch_size=16, epochs=20, validation_data=(x_dev, y_dev))
         batch_size=16, epochs=50, validation_data=(x_dev, y_dev), callbacks=callbacks)

preds = model.predict(x_dev, batch_size=16)

print()
metrics(preds, y_dev)

Train on 18975 samples, validate on 2108 samples
Epoch 1/50
18975/18975 [==============================] - 6s 335us/step - loss: 0.2253 - val_loss: 0.2392
Epoch 2/50
18975/18975 [==============================] - 6s 300us/step - loss: 0.2199 - val_loss: 0.2410
Epoch 3/50
18975/18975 [==============================] - 5s 278us/step - loss: 0.2197 - val_loss: 0.2401

Precision: 0
Recall: 0.0
F1: 0.0
Accuracy: 0.9354838709677419


# Hatebase lexicon attempts

In [46]:
# Load Hatebase lexicon
with open('/usr0/home/mamille2/11-830_data/project/hatebase_slurs.txt', 'r') as f:
# with open('../../project/hatebase_slurs.txt', 'r') as f:
    slurs = set([w.lower() for w in f.read().splitlines()])
    
len(slurs)

584

## Add as feature

In [47]:
# Extract Hatebase features

data['train']['num_slurs'] = data['train']['tweet_processed'].map(lambda x: sum([1 for wd in x.split() if wd in slurs]))
data['train']['num_slurs'].head()

0    1
1    0
2    1
3    1
4    0
Name: num_slurs, dtype: int64

In [50]:
vec = CountVectorizer(min_df=2, ngram_range=(1,2))
vec.fit(data['train'].tweet_processed.tolist())

bow = {}
bow['train'] = vec.transform(data['train'].tweet_processed.tolist())
bow['train'].shape

(21083, 34274)

In [51]:
# Hate speech classification
x = {}
y = {}
x['train'] = hstack([bow['train'], csr_matrix(data['train']['num_slurs'].data).T])
y['train'] = data['train'].hate

clf = LogisticRegression()

scoring = {'accuracy': make_scorer(accuracy_score), 
           'precision': make_scorer(precision_score),
          'recall': make_scorer(recall_score),
          'f1': make_scorer(f1_score)}

scores = cross_validate(clf, x['train'], y['train'], cv=5, scoring=scoring)
avg_prec = scores["test_precision"].mean()
avg_rec = scores["test_recall"].mean()

print(f'Accuracy: {scores["test_accuracy"].mean(): .1%}')
print(f'Precision: {avg_prec: .1%}')
print(f'Recall: {avg_rec: .1%}')
# print(f'F1: {scores["test_f1"].mean(): .1%}')
print(f'F1: {f1(avg_prec, avg_rec): .1%}')

Accuracy:  93.6%
Precision:  53.4%
Recall:  23.0%
F1:  32.1%


In [52]:
# Offensive language classification
x = {}
y = {}
x['train'] = hstack([bow['train'], csr_matrix(data['train']['num_slurs'].data).T])
y['train'] = data['train'].offensive

clf = LogisticRegression()

scoring = {'accuracy': make_scorer(accuracy_score), 
           'precision': make_scorer(precision_score),
          'recall': make_scorer(recall_score),
          'f1': make_scorer(f1_score)}

scores = cross_validate(clf, x['train'], y['train'], cv=5, scoring=scoring)
avg_prec = scores["test_precision"].mean()
avg_rec = scores["test_recall"].mean()

print(f'Accuracy: {scores["test_accuracy"].mean(): .1%}')
print(f'Precision: {avg_prec: .1%}')
print(f'Recall: {avg_rec: .1%}')
# print(f'F1: {scores["test_f1"].mean(): .1%}')
print(f'F1: {f1(avg_prec, avg_rec): .1%}')

Accuracy:  89.4%
Precision:  93.3%
Recall:  93.0%
F1:  93.2%


## Bigrams (with hate speech lexicon pairing and not)

In [42]:
# Normal bigrams

vec = CountVectorizer(ngram_range=(1,2), min_df=2)
vec.fit(data['train'].tweet_processed.tolist())

bow = {}
bow['train'] = vec.transform(data['train'].tweet_processed.tolist())
bow['train'].shape

(21083, 34274)

In [36]:
# Hate speech classification
x = {}
y = {}
x['train'] = bow['train']
y['train'] = data['train'].hate

clf = LogisticRegression()

scoring = {'accuracy': make_scorer(accuracy_score), 
           'precision': make_scorer(precision_score),
          'recall': make_scorer(recall_score),
          'f1': make_scorer(f1_score)}

scores = cross_validate(clf, x['train'], y['train'], cv=5, scoring=scoring)

avg_prec = scores["test_precision"].mean()
avg_rec = scores["test_recall"].mean()

print(f'Accuracy: {scores["test_accuracy"].mean(): .1%}')
print(f'Precision: {avg_prec: .1%}')
print(f'Recall: {avg_rec: .1%}')
# print(f'F1: {scores["test_f1"].mean(): .1%}')
print(f'F1: {f1(avg_prec, avg_rec): .1%}')

Accuracy:  93.7%
Precision:  52.9%
Recall:  22.7%
F1:  31.7%


In [43]:
# Offensive language classification
x = {}
y = {}
x['train'] = bow['train']
y['train'] = data['train'].offensive

clf = LogisticRegression()

scoring = {'accuracy': make_scorer(accuracy_score), 
           'precision': make_scorer(precision_score),
          'recall': make_scorer(recall_score),
          'f1': make_scorer(f1_score)}

scores = cross_validate(clf, x['train'], y['train'], cv=5, scoring=scoring)
avg_prec = scores["test_precision"].mean()
avg_rec = scores["test_recall"].mean()

print(f'Accuracy: {scores["test_accuracy"].mean(): .1%}')
print(f'Precision: {avg_prec: .1%}')
print(f'Recall: {avg_rec: .1%}')
# print(f'F1: {scores["test_f1"].mean(): .1%}')
print(f'F1: {f1(avg_prec, avg_rec): .1%}')

Accuracy:  89.4%
Precision:  93.3%
Recall:  93.1%
F1:  93.2%


In [53]:
def get_hate_pairs(tweet):
    toks = tweet.split()
    hate_dict = defaultdict(int)
    
    hate_terms = [w for w in toks if w in slurs]
    for h in hate_terms:
        for t in toks:
            hate_dict[f'{h}_{t}'] += 1
        
    return hate_dict

In [116]:
# Extract dict features
hate_pairs = data['train']['tweet_processed'].map(get_hate_pairs)
hate_pairs[:10]

0    {'nigger_<mention>': 2, 'nigger_they': 2, 'nig...
1                                                   {}
2    {'faggot_<mention>': 1, 'faggot_sion': 1, 'fag...
3    {'redneck_<mention>': 3, 'redneck_your': 1, 'r...
4                                                   {}
5                                                   {}
6                                                   {}
7    {'whitey_<mention>': 1, 'whitey_ok': 1, 'white...
8    {'faggot_<mention>': 1, 'faggot_shut': 1, 'fag...
9                                                   {}
Name: tweet_processed, dtype: object

In [117]:
# Build special pairings with hatebase terms
dict_vec = DictVectorizer()

hate_feats = dict_vec.fit_transform(hate_pairs)
hate_feats.shape

(21083, 57450)

In [62]:
vec = CountVectorizer(min_df=2, ngram_range=(1,2))
# vec = CountVectorizer(min_df=2)
vec.fit(data['train'].tweet_processed.tolist())

bow = {}
bow['train'] = vec.transform(data['train'].tweet_processed.tolist())
bow['train'].shape

(21083, 34274)

In [63]:
# Hate speech classification
x = {}
y = {}
x['train'] = hstack([bow['train'], hate_feats])
y['train'] = data['train'].hate

# clf = make_pipeline(SelectKBest(chi2, k=30000), LogisticRegression())
clf = LogisticRegression()

scoring = {'accuracy': make_scorer(accuracy_score), 
           'precision': make_scorer(precision_score),
          'recall': make_scorer(recall_score),
          'f1': make_scorer(f1_score)}

scores = cross_validate(clf, x['train'], y['train'], cv=5, scoring=scoring)
avg_prec = scores["test_precision"].mean()
avg_rec = scores["test_recall"].mean()

print(f'Accuracy: {scores["test_accuracy"].mean(): .1%}')
print(f'Precision: {avg_prec: .1%}')
print(f'Recall: {avg_rec: .1%}')
# print(f'F1: {scores["test_f1"].mean(): .1%}')
print(f'F1: {f1(avg_prec, avg_rec): .1%}')

Accuracy:  93.5%
Precision:  50.7%
Recall:  23.1%
F1:  31.7%


In [64]:
# Offensive language classification
x = {}
y = {}
x['train'] = hstack([bow['train'], hate_feats])
y['train'] = data['train'].offensive

clf = LogisticRegression()

scoring = {'accuracy': make_scorer(accuracy_score), 
           'precision': make_scorer(precision_score),
          'recall': make_scorer(recall_score),
          'f1': make_scorer(f1_score)}

scores = cross_validate(clf, x['train'], y['train'], cv=5, scoring=scoring)
avg_prec = scores["test_precision"].mean()
avg_rec = scores["test_recall"].mean()

print(f'Accuracy: {scores["test_accuracy"].mean(): .1%}')
print(f'Precision: {avg_prec: .1%}')
print(f'Recall: {avg_rec: .1%}')
# print(f'F1: {scores["test_f1"].mean(): .1%}')
print(f'F1: {f1(avg_prec, avg_rec): .1%}')

Accuracy:  90.0%
Precision:  93.9%
Recall:  93.1%
F1:  93.5%


## Violence words

In [65]:
viol_wds = ['kill', 'murder', 'stab', 'death', 'bullets', 'blood', 'knife', 'gun', 'shoot', 'life', 'beat', 'hate',
           'fight']

In [72]:
# Extract Hatebase features

data['train']['num_viol'] = data['train']['tweet_processed'].map(lambda x: sum([1 for wd in x.split() if wd in viol_wds]))
data['train']['num_viol'].head()

0    0
1    0
2    0
3    0
4    0
Name: num_viol, dtype: int64

In [73]:
vec = CountVectorizer(min_df=2, ngram_range=(1,2))
vec.fit(data['train'].tweet_processed.tolist())

bow = {}
bow['train'] = vec.transform(data['train'].tweet_processed.tolist())
bow['train'].shape

(21083, 34274)

In [75]:
# Hate speech classification
x = {}
y = {}
x['train'] = hstack([bow['train'], csr_matrix(data['train']['num_viol'].data).T])
y['train'] = data['train'].hate

clf = LogisticRegression()

scoring = {'accuracy': make_scorer(accuracy_score), 
           'precision': make_scorer(precision_score),
          'recall': make_scorer(recall_score),
          'f1': make_scorer(f1_score)}

scores = cross_validate(clf, x['train'], y['train'], cv=5, scoring=scoring)
avg_prec = scores["test_precision"].mean()
avg_rec = scores["test_recall"].mean()

print(f'Accuracy: {scores["test_accuracy"].mean(): .1%}')
print(f'Precision: {avg_prec: .1%}')
print(f'Recall: {avg_rec: .1%}')
# print(f'F1: {scores["test_f1"].mean(): .1%}')
print(f'F1: {f1(avg_prec, avg_rec): .1%}')

Accuracy:  93.6%
Precision:  52.8%
Recall:  22.7%
F1:  31.8%


In [76]:
# Offensive language classification
x = {}
y = {}
x['train'] = hstack([bow['train'], csr_matrix(data['train']['num_viol'].data).T])
y['train'] = data['train'].offensive

clf = LogisticRegression()

scoring = {'accuracy': make_scorer(accuracy_score), 
           'precision': make_scorer(precision_score),
          'recall': make_scorer(recall_score),
          'f1': make_scorer(f1_score)}

scores = cross_validate(clf, x['train'], y['train'], cv=5, scoring=scoring)
avg_prec = scores["test_precision"].mean()
avg_rec = scores["test_recall"].mean()

print(f'Accuracy: {scores["test_accuracy"].mean(): .1%}')
print(f'Precision: {avg_prec: .1%}')
print(f'Recall: {avg_rec: .1%}')
# print(f'F1: {scores["test_f1"].mean(): .1%}')
print(f'F1: {f1(avg_prec, avg_rec): .1%}')

Accuracy:  89.4%
Precision:  93.3%
Recall:  93.1%
F1:  93.2%


In [66]:
def get_viol_pairs(tweet):
    toks = tweet.split()
    hate_dict = defaultdict(int)
    
    hate_terms = [w for w in toks if w in viol_wds]
    for h in hate_terms:
        for t in toks:
            hate_dict[f'{h}_{t}'] += 1
        
    return hate_dict

In [67]:
# Extract dict features
hate_pairs = data['train']['tweet_processed'].map(get_viol_pairs)
hate_pairs[:10]

0    {}
1    {}
2    {}
3    {}
4    {}
5    {}
6    {}
7    {}
8    {}
9    {}
Name: tweet_processed, dtype: object

In [117]:
len([h for h in hate_pairs.tolist() if len(h) > 0])

1094

In [68]:
# Build special pairings with hatebase terms
dict_vec = DictVectorizer()

hate_feats = dict_vec.fit_transform(hate_pairs)
hate_feats.shape

(21083, 6339)

In [69]:
vec = CountVectorizer(min_df=2, ngram_range=(1,2))
vec.fit(data['train'].tweet_processed.tolist())

bow = {}
bow['train'] = vec.transform(data['train'].tweet_processed.tolist())
bow['train'].shape

(21083, 34274)

In [70]:
# Hate speech classification
x = {}
y = {}
x['train'] = hstack([bow['train'], hate_feats])
# x['train'] = bow['train']
y['train'] = data['train'].hate

clf = make_pipeline(SelectKBest(chi2, k=10000), LogisticRegression())
clf = LogisticRegression()

scoring = {'accuracy': make_scorer(accuracy_score), 
           'precision': make_scorer(precision_score),
          'recall': make_scorer(recall_score),
          'f1': make_scorer(f1_score)}

scores = cross_validate(clf, x['train'], y['train'], cv=5, scoring=scoring)
avg_prec = scores["test_precision"].mean()
avg_rec = scores["test_recall"].mean()

print(f'Accuracy: {scores["test_accuracy"].mean(): .1%}')
print(f'Precision: {avg_prec: .1%}')
print(f'Recall: {avg_rec: .1%}')
# print(f'F1: {scores["test_f1"].mean(): .1%}')
print(f'F1: {f1(avg_prec, avg_rec): .1%}')

Accuracy:  93.6%
Precision:  51.7%
Recall:  22.8%
F1:  31.7%


In [71]:
# Offensive language classification
x = {}
y = {}
x['train'] = hstack([bow['train'], hate_feats])
y['train'] = data['train'].offensive

clf = LogisticRegression()

scoring = {'accuracy': make_scorer(accuracy_score), 
           'precision': make_scorer(precision_score),
          'recall': make_scorer(recall_score),
          'f1': make_scorer(f1_score)}

scores = cross_validate(clf, x['train'], y['train'], cv=5, scoring=scoring)
avg_prec = scores["test_precision"].mean()
avg_rec = scores["test_recall"].mean()

print(f'Accuracy: {scores["test_accuracy"].mean(): .1%}')
print(f'Precision: {avg_prec: .1%}')
print(f'Recall: {avg_rec: .1%}')
# print(f'F1: {scores["test_f1"].mean(): .1%}')
print(f'F1: {f1(avg_prec, avg_rec): .1%}')

Accuracy:  89.3%
Precision:  93.3%
Recall:  93.0%
F1:  93.1%


## POS templates

In [77]:
data['train']['pos'] = data['train']['tweet_processed'].map(lambda x: ' '.join([pos for wd, pos in pos_tag(x.split())]))
data['train']['pos'].head()

0    NNS VBP PRP VBP WP NN VBD , VB VBP WP PRP VBD ...
1                    JJ NN NN NN FW RB IN DT NNS VBP .
2    JJ NN VBZ DT NN . RB JJ NNS VBP : CD CC VB : C...
3    JJ NNP NNP PRP$ JJ NN NN NNS VBP IN DT JJ NNS ...
4                            RB VB IN IN PRP$ NN NNS .
Name: pos, dtype: object

In [78]:
def pos_contexts(text, postags, window_size):
    toks = text.split()
    tags = postags.split()
    feats = []
    
    assert len(toks) == len(tags)
    
    for i in range(len(toks)):
        if i == 0:
            feats.append('_'.join([toks[i]] + tags[i+1:i+1+window_size]))
        elif i == len(toks) - 1:
            feats.append('_'.join(tags[i-window_size:i] + [tags[i]]))
        else:
            feats.append('_'.join(tags[i-window_size:i] + [toks[i]] + tags[i+1:i+1+window_size]))
    
    return Counter(feats)

In [79]:
pos_templates = [pos_contexts(t, postags, 1) for (t, postags) in list(zip(data['train']['tweet_processed'], data['train']['pos']))]
len(pos_templates)

21083

In [80]:
# Build special pairings with hatebase terms
dict_vec = DictVectorizer()

hate_feats = dict_vec.fit_transform(pos_templates)
hate_feats.shape

(21083, 106507)

In [81]:
vec = CountVectorizer(min_df=2, ngram_range=(1,2))
vec.fit(data['train'].tweet_processed.tolist())

bow = {}
bow['train'] = vec.transform(data['train'].tweet_processed.tolist())
bow['train'].shape

(21083, 34274)

In [85]:
# Hate speech classification
x = {}
y = {}
x['train'] = hstack([bow['train'], hate_feats])
y['train'] = data['train'].hate

# clf = make_pipeline(SelectKBest(chi2, k=10000), LogisticRegression())
clf = LogisticRegression()

scoring = {'accuracy': make_scorer(accuracy_score), 
           'precision': make_scorer(precision_score),
          'recall': make_scorer(recall_score),
          'f1': make_scorer(f1_score)}

scores = cross_validate(clf, x['train'], y['train'], cv=5, scoring=scoring)
avg_prec = scores["test_precision"].mean()
avg_rec = scores["test_recall"].mean()

print(f'Accuracy: {scores["test_accuracy"].mean(): .1%}')
print(f'Precision: {avg_prec: .1%}')
print(f'Recall: {avg_rec: .1%}')
# print(f'F1: {scores["test_f1"].mean(): .1%}')
print(f'F1: {f1(avg_prec, avg_rec): .1%}')

Accuracy:  90.6%
Precision:  43.4%
Recall:  21.5%
F1:  28.8%


In [88]:
# Offensive language classification
x = {}
y = {}
x['train'] = hstack([bow['train'], hate_feats])
y['train'] = data['train'].offensive

clf = LogisticRegression()

scoring = {'accuracy': make_scorer(accuracy_score), 
           'precision': make_scorer(precision_score),
          'recall': make_scorer(recall_score),
          'f1': make_scorer(f1_score)}

scores = cross_validate(clf, x['train'], y['train'], cv=5, scoring=scoring)
avg_prec = scores["test_precision"].mean()
avg_rec = scores["test_recall"].mean()

print(f'Accuracy: {scores["test_accuracy"].mean(): .1%}')
print(f'Precision: {avg_prec: .1%}')
print(f'Recall: {avg_rec: .1%}')
# print(f'F1: {scores["test_f1"].mean(): .1%}')
print(f'F1: {f1(avg_prec, avg_rec): .1%}')

Accuracy:  88.7%
Precision:  93.6%
Recall:  91.7%
F1:  92.7%


## POS ngrams (doesn't improve)

In [96]:
data['train']['pos'] = data['train']['tweet_processed'].map(lambda x: ' '.join([pos for wd, pos in pos_tag(x.split())]))
data['train']['pos'].head()

0    NNS VBP PRP VBP WP NN VBD , VB VBP WP PRP VBD , NN VBD DT NN NN , NN VBP PRP TO                   
1    JJ NN NN NN FW RB IN DT NNS VBP .                                                                 
2    JJ NN VBZ DT NN . RB JJ NNS VBP : CD CC VB : CD PRP$ NNS VBP RB RB JJ NN RB DT NN , NN JJ NN NN NN
3    JJ NNP NNP PRP$ JJ NN NN NNS VBP IN DT JJ NNS IN NNS IN PRP VBP NNS                               
4    RB VB IN IN PRP$ NN NNS .                                                                         
Name: pos, dtype: object

In [89]:
# Extract ngrams
pos_vec = CountVectorizer(ngram_range=(1,3))
pos_feats = pos_vec.fit_transform(data['train']['pos'])
pos_feats.shape

(21083, 8972)

In [91]:
# Hate speech classification
x = {}
y = {}
x['train'] = hstack([bow['train'], pos_feats])
y['train'] = data['train'].hate

# clf = make_pipeline(SelectKBest(chi2, k=10000), LogisticRegression())
clf = LogisticRegression()

scoring = {'accuracy': make_scorer(accuracy_score), 
           'precision': make_scorer(precision_score),
          'recall': make_scorer(recall_score),
          'f1': make_scorer(f1_score)}

scores = cross_validate(clf, x['train'], y['train'], cv=5, scoring=scoring)
avg_prec = scores["test_precision"].mean()
avg_rec = scores["test_recall"].mean()

print(f'Accuracy: {scores["test_accuracy"].mean(): .1%}')
print(f'Precision: {avg_prec: .1%}')
print(f'Recall: {avg_rec: .1%}')
# print(f'F1: {scores["test_f1"].mean(): .1%}')
print(f'F1: {f1(avg_prec, avg_rec): .1%}')

Accuracy:  93.1%
Precision:  45.6%
Recall:  22.2%
F1:  29.8%


In [92]:
# Offensive language classification
x = {}
y = {}
x['train'] = hstack([bow['train'], pos_feats])
y['train'] = data['train'].offensive

clf = LogisticRegression()

scoring = {'accuracy': make_scorer(accuracy_score), 
           'precision': make_scorer(precision_score),
          'recall': make_scorer(recall_score),
          'f1': make_scorer(f1_score)}

scores = cross_validate(clf, x['train'], y['train'], cv=5, scoring=scoring)
avg_prec = scores["test_precision"].mean()
avg_rec = scores["test_recall"].mean()

print(f'Accuracy: {scores["test_accuracy"].mean(): .1%}')
print(f'Precision: {avg_prec: .1%}')
print(f'Recall: {avg_rec: .1%}')
# print(f'F1: {scores["test_f1"].mean(): .1%}')
print(f'F1: {f1(avg_prec, avg_rec): .1%}')

Accuracy:  88.3%
Precision:  92.7%
Recall:  92.3%
F1:  92.5%


## Hate density (doesn't improve)

In [93]:
def slurs_norm(num_slurs, text):
    return num_slurs/len(text.split())

In [94]:
data['train']['slurs_normalized'] = [slurs_norm(ns, t) for (ns, t) in list(zip(data['train']['num_slurs'], data['train']['tweet_processed']))]
data['train']['slurs_normalized'].head()

0    0.041667
1    0.000000
2    0.031250
3    0.052632
4    0.000000
Name: slurs_normalized, dtype: float64

In [95]:
# Hate speech classification
x = {}
y = {}
x['train'] = hstack([bow['train'], csr_matrix(data['train']['slurs_normalized'].data).T])
y['train'] = data['train'].hate

clf = LogisticRegression()

scoring = {'accuracy': make_scorer(accuracy_score), 
           'precision': make_scorer(precision_score),
          'recall': make_scorer(recall_score),
          'f1': make_scorer(f1_score)}

scores = cross_validate(clf, x['train'], y['train'], cv=5, scoring=scoring)
avg_prec = scores["test_precision"].mean()
avg_rec = scores["test_recall"].mean()

print(f'Accuracy: {scores["test_accuracy"].mean(): .1%}')
print(f'Precision: {avg_prec: .1%}')
print(f'Recall: {avg_rec: .1%}')
# print(f'F1: {scores["test_f1"].mean(): .1%}')
print(f'F1: {f1(avg_prec, avg_rec): .1%}')

Accuracy:  93.7%
Precision:  53.3%
Recall:  22.9%
F1:  32.1%


In [96]:
# Offensive language classification
x = {}
y = {}
x['train'] = hstack([bow['train'], csr_matrix(data['train']['slurs_normalized'].data).T])
y['train'] = data['train'].offensive

clf = LogisticRegression()

scoring = {'accuracy': make_scorer(accuracy_score), 
           'precision': make_scorer(precision_score),
          'recall': make_scorer(recall_score),
          'f1': make_scorer(f1_score)}

scores = cross_validate(clf, x['train'], y['train'], cv=5, scoring=scoring)
avg_prec = scores["test_precision"].mean()
avg_rec = scores["test_recall"].mean()

print(f'Accuracy: {scores["test_accuracy"].mean(): .1%}')
print(f'Precision: {avg_prec: .1%}')
print(f'Recall: {avg_rec: .1%}')
# print(f'F1: {scores["test_f1"].mean(): .1%}')
print(f'F1: {f1(avg_prec, avg_rec): .1%}')

Accuracy:  89.5%
Precision:  93.3%
Recall:  93.1%
F1:  93.2%


# Sentiment analysis

In [5]:
sid = SentimentIntensityAnalyzer()

In [6]:
sent_feats = np.array(data['train']['tweet_processed'].map(lambda x: list(sid.polarity_scores(x).values())).tolist())
sent_feats = np.array(sent_feats)
sent_feats.shape

(21083, 4)

In [104]:
# Hate speech classification
x = {}
y = {}
x['train'] = hstack([bow['train'], csr_matrix(sent_feats)])
y['train'] = data['train'].hate

clf = LogisticRegression()

scoring = {'accuracy': make_scorer(accuracy_score), 
           'precision': make_scorer(precision_score),
          'recall': make_scorer(recall_score),
          'f1': make_scorer(f1_score)}

scores = cross_validate(clf, x['train'], y['train'], cv=5, scoring=scoring)
avg_prec = scores["test_precision"].mean()
avg_rec = scores["test_recall"].mean()

print(f'Accuracy: {scores["test_accuracy"].mean(): .1%}')
print(f'Precision: {avg_prec: .1%}')
print(f'Recall: {avg_rec: .1%}')
# print(f'F1: {scores["test_f1"].mean(): .1%}')
print(f'F1: {f1(avg_prec, avg_rec): .1%}')

Accuracy:  93.7%
Precision:  53.3%
Recall:  22.9%
F1:  32.1%


In [105]:
# Offensive language classification
x = {}
y = {}
x['train'] = hstack([bow['train'], csr_matrix(sent_feats)])
y['train'] = data['train'].offensive

clf = LogisticRegression()

scoring = {'accuracy': make_scorer(accuracy_score), 
           'precision': make_scorer(precision_score),
          'recall': make_scorer(recall_score),
          'f1': make_scorer(f1_score)}

scores = cross_validate(clf, x['train'], y['train'], cv=5, scoring=scoring)
avg_prec = scores["test_precision"].mean()
avg_rec = scores["test_recall"].mean()

print(f'Accuracy: {scores["test_accuracy"].mean(): .1%}')
print(f'Precision: {avg_prec: .1%}')
print(f'Recall: {avg_rec: .1%}')
# print(f'F1: {scores["test_f1"].mean(): .1%}')
print(f'F1: {f1(avg_prec, avg_rec): .1%}')

Accuracy:  89.5%
Precision:  93.3%
Recall:  93.1%
F1:  93.2%


# Feature combinations (final model)

In [15]:
folds = ['train', 'test']

In [16]:
vec = CountVectorizer(min_df=2, ngram_range=(1,2))
vec.fit(data['train'].tweet_processed.tolist())

bow = {}
for f in folds:
    bow[f] = vec.transform(data[f].tweet_processed.tolist())
    print(bow[f].shape)

(21083, 34274)
(3700, 34274)


In [17]:
sid = SentimentIntensityAnalyzer()

In [18]:
sent_feats = {}

for f in ['train', 'test']:
    sent_feats[f] = np.array(data[f]['tweet_processed'].map(lambda x: list(sid.polarity_scores(x).values())).tolist())
    sent_feats[f] = np.array(sent_feats[f])
    print(sent_feats[f].shape)

(21083, 4)
(3700, 4)


In [19]:
# Load Hatebase lexicon
with open('/usr0/home/mamille2/11-830_data/project/hatebase_slurs.txt', 'r') as f:
# with open('../../project/hatebase_slurs.txt', 'r') as f:
    slurs = set([w.lower() for w in f.read().splitlines()])
    
print(len(slurs))

# Extract Hatebase features

for f in ['train', 'test']:
    data[f]['num_slurs'] = data[f]['tweet_processed'].map(lambda x: sum([1 for wd in x.split() if wd in slurs]))

584


In [20]:
viol_wds = ['kill', 'murder', 'stab', 'death', 'bullets', 'blood', 'knife', 'gun', 'shoot', 'life', 'beat', 'hate',
           'fight']

for f in ['train', 'test']:
    data[f]['num_viol'] = data[f]['tweet_processed'].map(lambda x: sum([1 for wd in x.split() if wd in viol_wds]))

In [21]:
def get_hate_pairs(tweet):
    toks = tweet.split()
    hate_dict = defaultdict(int)
    
    hate_terms = [w for w in toks if w in slurs]
    for h in hate_terms:
        for t in toks:
            hate_dict[f'{h}_{t}'] += 1
        
    return hate_dict

hate_pairs = {}

# Extract dict features
for f in ['train', 'test']:
    hate_pairs[f] = data[f]['tweet_processed'].map(get_hate_pairs)

# Build special pairings with hatebase terms
hate_feats = {}

# Fit on training
dict_vec = DictVectorizer()
dict_vec.fit(hate_pairs['train'])

for f in ['train', 'test']:

    hate_feats[f] = dict_vec.transform(hate_pairs[f])
    print(hate_feats[f].shape)

(21083, 57450)
(3700, 57450)


In [22]:
def f1(prec, rec):
    return 2*prec*rec / (prec + rec)

In [23]:
# Hate speech classification
x = {}
y = {}
x['train'] = hstack([bow['train'], csr_matrix(data['train']['num_slurs'].data).T, 
                     csr_matrix(data['train']['num_viol'].data).T,
                     csr_matrix(sent_feats['train'])])
y['train'] = data['train'].hate

clf = LogisticRegression()

scoring = {'accuracy': make_scorer(accuracy_score), 
           'precision': make_scorer(precision_score),
          'recall': make_scorer(recall_score),
          'f1': make_scorer(f1_score)}

scores = cross_validate(clf, x['train'], y['train'], cv=5, scoring=scoring)
avg_prec = scores["test_precision"].mean()
avg_rec = scores["test_recall"].mean()

print(f'Accuracy: {scores["test_accuracy"].mean(): .1%}')
print(f'Precision: {avg_prec: .1%}')
print(f'Recall: {avg_rec: .1%}')
# print(f'F1: {scores["test_f1"].mean(): .1%}')
print(f'F1: {f1(avg_prec, avg_rec): .1%}')

Accuracy:  93.7%
Precision:  53.8%
Recall:  23.1%
F1:  32.3%


In [24]:
# Fit classifier to all training data
clf = LogisticRegression()
clf.fit(x['train'], y['train'])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [25]:
preds = clf.predict(x['train'])
confusion_matrix(y['train'], preds)

array([[19847,    20],
       [  289,   927]])

In [33]:
# Make predictions on test set
x['test'] = hstack([bow['test'], csr_matrix(data['test']['num_slurs'].data).T, 
                     csr_matrix(data['test']['num_viol'].data).T,
                     csr_matrix(sent_feats['test'])])
data['test']['hate'] = clf.predict(x['test'])
data['test']

,ID,tweet,tweet_processed,num_slurs,num_viol,hate
0,0,!!! RT @mayasolovely: As a woman you shouldn't...,! ! ! rt <mention> : as a woman you shouldn't ...,1,0,0
1,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,! ! ! rt <mention> : boy dats cold ... tyga dw...,1,0,0
2,2,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,! ! ! rt <mention> dawg ! ! ! rt <mention> : y...,1,0,0
3,3,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,! ! ! rt <mention> : <mention> she look like a...,1,0,0
4,4,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,! ! ! rt <mention> : the shit you hear about m...,1,0,0
5,5,"!!!!!!!!!!!!!!!!!!""@T_Madison_x: The shit just...","! ! ! "" <mention> : the shit just blows me .. ...",0,0,0
6,6,"!!!!!!""@__BrighterDays: I can not just sit up ...","! ! ! "" <mention> : i can not just sit up and ...",1,1,0
7,7,!!!!&#8220;@selfiequeenbri: cause I'm tired of...,! ! ! “ <mention> : cause i'm tired of you big...,1,0,0
8,8,""" &amp; you might not get ya bitch back &amp; ...",""" & you might not get ya bitch back & thats th...",1,0,0
9,9,""" @rhythmixx_ :hobbies include: fighting Maria...",""" <mention> : hobbies include : fighting maria...",1,0,0


In [34]:
sum(data['test']['hate'])

111

In [122]:
# Hate speech classification
x = {}
y = {}
x['train'] = hstack([bow['train'], csr_matrix(data['train']['num_slurs'].data).T, 
                     csr_matrix(data['train']['num_viol'].data).T,
                     csr_matrix(sent_feats)])
y['train'] = data['train'].hate

clf = LogisticRegression()

scoring = {'accuracy': make_scorer(accuracy_score), 
           'precision': make_scorer(precision_score),
          'recall': make_scorer(recall_score),
          'f1': make_scorer(f1_score)}

scores = cross_validate(clf, x['train'], y['train'], cv=5, scoring=scoring)
avg_prec = scores["test_precision"].mean()
avg_rec = scores["test_recall"].mean()

print(f'Accuracy: {scores["test_accuracy"].mean(): .1%}')
print(f'Precision: {avg_prec: .1%}')
print(f'Recall: {avg_rec: .1%}')
# print(f'F1: {scores["test_f1"].mean(): .1%}')
print(f'F1: {f1(avg_prec, avg_rec): .1%}')

Accuracy:  93.7%
Precision:  53.8%
Recall:  23.1%
F1:  32.3%


In [26]:
# Offensive language classification
x = {}
y = {}
x['train'] = hstack([bow['train'], csr_matrix(data['train']['num_slurs'].data).T, 
                     csr_matrix(data['train']['num_viol'].data).T,
                     csr_matrix(sent_feats['train']),
                    hate_feats['train']])
y['train'] = data['train'].offensive

clf = LogisticRegression()

scoring = {'accuracy': make_scorer(accuracy_score), 
           'precision': make_scorer(precision_score),
          'recall': make_scorer(recall_score),
          'f1': make_scorer(f1_score)}

scores = cross_validate(clf, x['train'], y['train'], cv=5, scoring=scoring)
avg_prec = scores["test_precision"].mean()
avg_rec = scores["test_recall"].mean()

print(f'Accuracy: {scores["test_accuracy"].mean(): .1%}')
print(f'Precision: {avg_prec: .1%}')
print(f'Recall: {avg_rec: .1%}')
# print(f'F1: {scores["test_f1"].mean(): .1%}')
print(f'F1: {f1(avg_prec, avg_rec): .1%}')

Accuracy:  89.8%
Precision:  93.8%
Recall:  93.0%
F1:  93.4%


In [27]:
# Fit classifier to all training data
clf = LogisticRegression()
clf.fit(x['train'], y['train'])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [28]:
preds = clf.predict(x['train'])
confusion_matrix(y['train'], preds)

array([[ 4653,   105],
       [   44, 16281]])

In [43]:
x['train'].shape

(21083, 91730)

In [44]:
x['test'].shape

(3700, 50896)

In [45]:
for f in folds:
    print(bow[f].shape)

(21083, 34274)
(3700, 34274)


In [47]:
# Make predictions on test set
x['test'] = hstack([bow['test'], csr_matrix(data['test']['num_slurs'].data).T, 
                     csr_matrix(data['test']['num_viol'].data).T,
                     csr_matrix(sent_feats['test']),
                    hate_feats['test']])
data['test']['offensive'] = clf.predict(x['test'])
data['test']

,ID,tweet,tweet_processed,num_slurs,num_viol,hate,offensive
0,0,!!! RT @mayasolovely: As a woman you shouldn't...,! ! ! rt <mention> : as a woman you shouldn't ...,1,0,0,0
1,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,! ! ! rt <mention> : boy dats cold ... tyga dw...,1,0,0,1
2,2,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,! ! ! rt <mention> dawg ! ! ! rt <mention> : y...,1,0,0,1
3,3,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,! ! ! rt <mention> : <mention> she look like a...,1,0,0,0
4,4,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,! ! ! rt <mention> : the shit you hear about m...,1,0,0,1
5,5,"!!!!!!!!!!!!!!!!!!""@T_Madison_x: The shit just...","! ! ! "" <mention> : the shit just blows me .. ...",0,0,0,1
6,6,"!!!!!!""@__BrighterDays: I can not just sit up ...","! ! ! "" <mention> : i can not just sit up and ...",1,1,0,1
7,7,!!!!&#8220;@selfiequeenbri: cause I'm tired of...,! ! ! “ <mention> : cause i'm tired of you big...,1,0,0,1
8,8,""" &amp; you might not get ya bitch back &amp; ...",""" & you might not get ya bitch back & thats th...",1,0,0,1
9,9,""" @rhythmixx_ :hobbies include: fighting Maria...",""" <mention> : hobbies include : fighting maria...",1,0,0,1


In [51]:
# Select hate speech if both hate speech and offensive lang predicted
both_idx = data['test'][(data['test']['hate']==1) & (data['test']['offensive'])].index
len(both_idx)

0

In [50]:
for i in both_idx:
    data['test'].loc[i, 'offensive'] = 0

In [52]:
data['test']['neither'] = [1 if not a and not b else 0 for a,b in zip(data['test']['hate'], data['test']['offensive'])]
data['test']

,ID,tweet,tweet_processed,num_slurs,num_viol,hate,offensive,neither
0,0,!!! RT @mayasolovely: As a woman you shouldn't...,! ! ! rt <mention> : as a woman you shouldn't ...,1,0,0,0,1
1,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,! ! ! rt <mention> : boy dats cold ... tyga dw...,1,0,0,1,0
2,2,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,! ! ! rt <mention> dawg ! ! ! rt <mention> : y...,1,0,0,1,0
3,3,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,! ! ! rt <mention> : <mention> she look like a...,1,0,0,0,1
4,4,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,! ! ! rt <mention> : the shit you hear about m...,1,0,0,1,0
5,5,"!!!!!!!!!!!!!!!!!!""@T_Madison_x: The shit just...","! ! ! "" <mention> : the shit just blows me .. ...",0,0,0,1,0
6,6,"!!!!!!""@__BrighterDays: I can not just sit up ...","! ! ! "" <mention> : i can not just sit up and ...",1,1,0,1,0
7,7,!!!!&#8220;@selfiequeenbri: cause I'm tired of...,! ! ! “ <mention> : cause i'm tired of you big...,1,0,0,1,0
8,8,""" &amp; you might not get ya bitch back &amp; ...",""" & you might not get ya bitch back & thats th...",1,0,0,1,0
9,9,""" @rhythmixx_ :hobbies include: fighting Maria...",""" <mention> : hobbies include : fighting maria...",1,0,0,1,0


In [53]:
final_predictions = data['test'].drop(['tweet_processed', 'num_slurs', 'num_viol'], axis=1, inplace=False)
final_predictions

,ID,tweet,hate,offensive,neither
0,0,!!! RT @mayasolovely: As a woman you shouldn't...,0,0,1
1,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,0,1,0
2,2,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,0,1,0
3,3,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,0,0,1
4,4,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,0,1,0
5,5,"!!!!!!!!!!!!!!!!!!""@T_Madison_x: The shit just...",0,1,0
6,6,"!!!!!!""@__BrighterDays: I can not just sit up ...",0,1,0
7,7,!!!!&#8220;@selfiequeenbri: cause I'm tired of...,0,1,0
8,8,""" &amp; you might not get ya bitch back &amp; ...",0,1,0
9,9,""" @rhythmixx_ :hobbies include: fighting Maria...",0,1,0


In [54]:
# Save predictions
final_predictions.to_csv('/usr0/home/mamille2/11-830_data/hw3/Michael_Yoder.hypothesis', index=False)

## Error analysis (hate speech)

In [30]:
# Train on full dataset (hate speech) and see errors
clf = LogisticRegression()
x['train'] = hstack([bow['train'], csr_matrix(data['train']['num_slurs'].data).T, 
                     csr_matrix(data['train']['num_viol'].data).T,
                     csr_matrix(sent_feats['train'])])
y['train'] = data['train'].hate
clf.fit(x['train'], y['train'])
preds = clf.predict(x['train'])
preds.shape

(21083,)

In [31]:
false_pos = []
false_neg = []

for i, (gold, pred) in enumerate(zip(data['train'].hate.tolist(), preds)):
    if pred != gold:
        if gold == 0:
            false_pos.append(i)
        elif gold == 1:
            false_neg.append(i)
        else:
            "What???"
            
print(len(false_pos))
print(len(false_neg))

20
289


In [32]:
pd.set_option('display.max_colwidth', -1)

In [33]:
print("False negatives:")
display(data['train'].loc[false_neg, ['tweet_processed', 'hate', 'offensive', 'neither']].sample(30))

False negatives:


,tweet_processed,hate,offensive,neither
14338,rt <mention> : i hate a bitch dat act like a nigga . 😒,1,0,0
19518,you a hoe if you crying because yo ass pregnant . it was all smiles while you was riding that dick bitch .,1,0,0
10055,"pantera - 5 minutes alone http://t.co/srn5yc16 <mention> 5 minutes , pussy boy .. i'll crush you .",1,0,0
2383,<mention> nigger,1,0,0
3342,<mention> don't my nigga me honkie,1,0,0
6750,i hate dumb bitches,1,0,0
2165,<mention> suck my dick tacos el negro better,1,0,0
17199,smh hate that hoe,1,0,0
5556,fuck her pussy fuck her mouth fuck her ass cause she nasty,1,0,0
17857,the fucking royals bitch ! work !,1,0,0


In [36]:
print("False positives:")
display(data['train'].loc[false_pos, ['tweet_processed', 'hate', 'offensive', 'neither']])

False positives:


,tweet_processed,hate,offensive,neither
115,<mention> <mention> faggot,0,1,0
143,<mention> r u a faggot ?,0,1,0
273,<mention> fag,0,1,0
581,<mention> <mention> faggot,0,1,0
728,<mention> you like men faggot .,0,1,0
772,<mention> shut up faggot,0,1,0
1434,<mention> faggot,0,1,0
1471,<mention> <mention> hey faggot spell faggot right you dipshit,0,1,0
1897,<mention> <mention> faggot,0,1,0
2059,"<mention> <mention> why wld u wanna be a faggot , dude ? that's faggot like",0,1,0


## Error analysis (offensive language)

In [37]:
# Train on full dataset and see errors
clf = LogisticRegression()
x['train'] = hstack([bow['train'], csr_matrix(data['train']['num_slurs'].data).T, 
                     csr_matrix(data['train']['num_viol'].data).T,
                     csr_matrix(sent_feats['train']),
                    hate_feats['train']])
y['train'] = data['train'].offensive
clf.fit(x['train'], y['train'])
preds = clf.predict(x['train'])
preds.shape

(21083,)

In [38]:
false_pos = []
false_neg = []

for i, (gold, pred) in enumerate(zip(data['train'].offensive.tolist(), preds)):
    if pred != gold:
        if gold == 0:
            false_pos.append(i)
        elif gold == 1:
            false_neg.append(i)
        else:
            "What???"
            
print(len(false_pos))
print(len(false_neg))

105
44


In [39]:
print("False negatives:")
display(data['train'].loc[false_neg, ['tweet_processed', 'hate', 'offensive', 'neither']].sample(30))

False negatives:


,tweet_processed,hate,offensive,neither
2001,<mention> fuck the yankees,0,1,0
11982,rt <mention> : meth brownies,0,1,0
115,<mention> <mention> faggot,0,1,0
1897,<mention> <mention> faggot,0,1,0
8553,know why birds don't wear pants ? cause their pecker's in the way,0,1,0
1217,<mention> stfu trash,0,1,0
18836,we trash,0,1,0
2461,<mention> faggot,0,1,0
18736,watch : excited isis jihadis getting their yezidi sex slaves on ' distribution day ' http://t.co/n9exnuvhks #tcot #lnyhbt #pjnet #tgdn #gop,0,1,0
20374,im weeaboo trash,0,1,0


In [40]:
print("False positives:")
display(data['train'].loc[false_pos, ['tweet_processed', 'hate', 'offensive', 'neither']].sample(30))

False positives:


,tweet_processed,hate,offensive,neither
5344,faggot ass bitch nigga,1,0,0
17489,stuck up serious bitches are possibly the worst type of girls . ever .,1,0,0
12906,rt <mention> : niggas talk more than bitches these days .,1,0,0
16770,raunchy ass hoe .,1,0,0
10279,"rt "" <mention> : & alla my niggas hot boys , so don't bring no ice in this bitch 💂 ”",1,0,0
9681,no girls no hoes just me myself and i,0,0,1
20135,"fuck ray rice , someone crack that bitch",1,0,0
6359,i be waking up on some new shit like fuck that bitch shit,1,0,0
8595,lrg ass nigguh,1,0,0
10551,rt <mention> : ray j is the perfect example of what happens when you give a lame nigga some pussy .,1,0,0
